# Consolidate data from seven counties data file  

Data for Counties:  
005 Bronx  
047 Kings  
059 Nassau  
061 NYC  
081 Queens  
085 Richmond   
103 Suffolk (LI)  


Parameter Codes used:  
88101 - PM2.5  
44201 - Ozone  
88502 - Acceptable PM2.5 AQI & Speciation Mass  
42401 - Sulfer Dioxide  
42602 - Nitrogen Dioxide  
42101 - Carbon monoxide    
81102 - PM10   

   


In [32]:
import pandas as pd

In [33]:
# Read 7 counties data (all columns filled)
seven_ny_counties_filled_df = pd.read_csv("Resources/Seven_NY_Counties_Filled.csv")

In [34]:
seven_ny_counties_filled_df.shape

(583581, 17)

In [35]:
# Sort by county_code & date_local
seven_ny_counties_filled_df = seven_ny_counties_filled_df.sort_values(['county_code', 'date_local'], \
                                                                      ascending=[True, True]).reset_index()

In [36]:
# Value counts for "county_code"
seven_ny_counties_filled_df['county_code'].value_counts()

county_code
5      179805
81     147349
103     83982
61      80442
85      34801
47      29179
59      28023
Name: count, dtype: int64

In [37]:
# Value counts for "parameter_code"
seven_ny_counties_filled_df['parameter_code'].value_counts()

parameter_code
88101    204161
44201    124081
88502     96665
42401     94275
42602     31826
42101     31159
81102      1414
Name: count, dtype: int64

In [38]:
county_list = ['005','047','059','061','081','085','103']
param_list =  ['42101','42401','44201','42602', '88101','88502','81102']

In [39]:
#Create dataframes for each county
county_dfs = {f"county_{county}": seven_ny_counties_filled_df[seven_ny_counties_filled_df['county_code'] == int(county)] 
              for county in county_list}

In [40]:
#Split each county dataframe by parameter_code
param_dfs = {f"{county}_{param}_df": county_dfs[f"county_{county}"].loc[county_dfs[f"county_{county}"]["parameter_code"] == int(param)]
             for county in county_list
             for param in param_list}

In [41]:

# Apply groupby 'date_local' and aggregation to each parameter dataframe
agg_dict = {'county_code': 'first',
            'parameter_code': 'first', 
            'parameter': 'first',
            'latitude': 'first',
            'longitude': 'first',
            'sample_duration_code': 'first',
            'units_of_measure': 'first',
            'observation_count': 'sum',
            'validity_indicator': 'first',
            'arithmetic_mean': 'mean',
            'first_max_value': 'mean',
            'first_max_hour': 'first',
            'aqi': 'mean',
            'county': 'first',
            'city': 'first' }


grouped_dfs = {key: df.groupby('date_local').agg(agg_dict).reset_index() 
               for key, df in param_dfs.items() if not df.empty}


In [42]:
# Join dataframes for each county
# One record per date_local with multiple parameters listed along same row

# Initialize dataframe
county_joined_dfs = {}

#Loop at county_list
for county in county_list:
    county_params = [f"{county}_{param}_df" for param in param_list]
    dfs_to_join = [grouped_dfs[key] for key in county_params if key in grouped_dfs]
    
    if dfs_to_join:
        joined_df = dfs_to_join[0]
        for df in dfs_to_join[1:]:
            joined_df = pd.merge(joined_df, df, on='date_local', how='outer', suffixes=('', f'_{df["parameter_code"].iloc[0]}'))
        county_joined_dfs[county] = joined_df

# Build final dataframe by combining all counties dataframes 
final_df = pd.concat(county_joined_dfs.values(), ignore_index=True)

In [43]:
# Add new column for Max AQI value
final_df['aqi_max'] = final_df[['aqi', 'aqi_42401', 'aqi_44201', 'aqi_42602', 'aqi_88101', 'aqi_88502', 'aqi_81102']].max(axis=1)

In [44]:
final_df.head()

,date_local,county_code,parameter_code,parameter,latitude,longitude,sample_duration_code,units_of_measure,observation_count,validity_indicator,...,units_of_measure_81102,observation_count_81102,validity_indicator_81102,arithmetic_mean_81102,first_max_value_81102,first_max_hour_81102,aqi_81102,county_81102,city_81102,aqi_max
0,2013-01-01,5.0,42101.0,Carbon monoxide,40.8679,-73.87809,1,Parts per million,43.0,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.0
1,2013-01-02,5.0,42101.0,Carbon monoxide,40.8679,-73.87809,1,Parts per million,48.0,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0
2,2013-01-03,5.0,42101.0,Carbon monoxide,40.8679,-73.87809,1,Parts per million,47.0,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.0
3,2013-01-04,5.0,42101.0,Carbon monoxide,40.8679,-73.87809,1,Parts per million,48.0,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.5
4,2013-01-05,5.0,42101.0,Carbon monoxide,40.8679,-73.87809,1,Parts per million,48.0,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.5


In [45]:
final_df.tail()

,date_local,county_code,parameter_code,parameter,latitude,longitude,sample_duration_code,units_of_measure,observation_count,validity_indicator,...,units_of_measure_81102,observation_count_81102,validity_indicator_81102,arithmetic_mean_81102,first_max_value_81102,first_max_hour_81102,aqi_81102,county_81102,city_81102,aqi_max
28731,2024-03-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.083333
28732,2024-03-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.250000
28733,2024-03-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.500000
28734,2024-03-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.333333
28735,2024-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.000000


In [46]:
# Verify new column aqi_max values in few records
aqi_values = final_df[['county_code', 'date_local','aqi', 'aqi_42401', 'aqi_44201',\
                       'aqi_42602', 'aqi_88101', 'aqi_88502', 'aqi_81102', 'aqi_max']][1:10]
aqi_values

,county_code,date_local,aqi,aqi_42401,aqi_44201,aqi_42602,aqi_88101,aqi_88502,aqi_81102,aqi_max
1,5.0,2013-01-02,6.5,11.4,21.000,26.5,NaN,60.0,NaN,60.0
2,5.0,2013-01-03,6.5,15.0,19.375,35.5,NaN,59.0,NaN,59.0
3,5.0,2013-01-04,7.0,13.0,19.000,36.5,64.5,60.0,NaN,64.5
4,5.0,2013-01-05,8.0,10.2,23.500,41.5,NaN,NaN,NaN,41.5
5,5.0,2013-01-06,9.0,13.0,19.750,34.0,NaN,NaN,NaN,34.0
6,5.0,2013-01-07,9.0,16.2,22.125,51.0,57.0,NaN,NaN,57.0
7,5.0,2013-01-08,9.5,18.2,13.000,44.0,NaN,60.0,NaN,60.0
8,5.0,2013-01-09,12.5,33.8,15.500,77.0,NaN,60.0,NaN,77.0
9,5.0,2013-01-10,10.0,10.2,20.500,36.0,35.5,54.5,NaN,54.5


In [47]:
final_df.shape

(28736, 107)

In [48]:
#display options to show all rows
pd.set_option('display.max_rows', None)

In [49]:
# Get null values fields count
final_df.isnull().sum()

date_local                        0
county_code                    7180
parameter_code                 7180
parameter                      7180
latitude                       7180
longitude                      7180
sample_duration_code           7180
units_of_measure               7180
observation_count              7180
validity_indicator             7180
arithmetic_mean                7180
first_max_value                7180
first_max_hour                 7180
aqi                            7180
county                         7180
city                           7180
county_code_42401             17756
parameter_code_42401          17756
parameter_42401               17756
latitude_42401                17756
longitude_42401               17756
sample_duration_code_42401    17756
units_of_measure_42401        17756
observation_count_42401       17756
validity_indicator_42401      17756
arithmetic_mean_42401         17756
first_max_value_42401         17756
first_max_hour_42401        

In [50]:
# List total records by county
print(final_df['county_code'].value_counts())

county_code
81.0     4101
5.0      4085
61.0     4049
59.0     4032
85.0     2872
47.0     1451
103.0     966
Name: count, dtype: int64


In [ ]:
# Write final output file to /Resources folder
final_df.to_csv("Resources/consolidated_seven_ny_counties.csv", index=False)